# Clasificación de imágenes con Redes Neuronales Convolucionales: Comparativa de tiempos de entrenamiento entre una GPU y CPU


* El objetivo de este notebook es el mostrar cuantitativamente la diferencia de tiempos en entrenar un modelo de deep learning con una CPU y una GPU.

* Para esta prueba vamos a disponer del siguiente Hardware:

    + CPU: intel core i7 10750h / 2.6 ghz
    + GPU: NVIDIA GeForce RTX 2060 8GB
    
* A continuación la implementación y ejecución:

## 1.- Activamos uso GPU y limitamos uso uso de memoria

In [1]:
import tensorflow.keras
import tensorflow as tf

from tensorflow.python.client import device_lib

# Limitación la memoria de la GPU
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.6
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# Permitir crecimiento de la memoria
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

## 2.- Obtenemos información de la GPU y la versión de TensorFlow

In [2]:
print('#### INFORMACIÓN ####')
print('  Versión de TensorFlow: {}'.format(tensorflow.__version__))
print('  GPU: {}'.format([x.physical_device_desc for x in device_lib.list_local_devices() if x.device_type == 'GPU']))
print('  Versión Cuda  -> {}'.format(tensorflow.sysconfig.get_build_info()['cuda_version']))
print('  Versión Cudnn -> {}\n'.format(tensorflow.sysconfig.get_build_info()['cudnn_version']))

#### INFORMACIÓN ####
  Versión de TensorFlow: 2.7.0
  GPU: ['device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5']
  Versión Cuda  -> 64_112
  Versión Cudnn -> 64_8



## 3.- Cargamos las imagenes de Entrenamiento y Test


* El dataset de imágenes se ha obtenido de Kaggle: https://www.kaggle.com/alaanagy/8-kinds-of-image-classification


* Para poder ejecutar este notebook se debe de descargar este dataset y guardarlo en la carpeta "data"

* Este dataset contiene 3 carpetas (pred, test  y train) con 35.000 imagenes clasificadas en 8 clases diferentes: seas, streets, buildings, glaciers, mountains, forests, cats, and dogs.


* Cada las carpetas test y train que son las que vamos a usar, contienen a su vez otras 8 carpetas; una por cada categoría, donde en cada una de esas carpetas estan las imágenes clasificadas por su categoría.


* Para este experimento, vamos a crearnos dos objetos de la clase ***ImageDataGenerator*** (https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator), que dada una carpeta (en nuestro caso la carpeta train y test) generará tantas imágenes como le indiquemos para el entrenamiento (y test o validación) del modelo a entrenar.


* Decimos tantas imágenes como le indiquemos ya que la clase ***ImageDataGenerator*** permite generar nuevas imágenes a partir de una dada haciendo ciertas modificaciones como rotaciones o zooms.


* Para este experimento vamos a crear 2 datasets de imágenes:

    + ***train_generator***: a partir de las carpeta de las imágenes de train, redimensionará las ***imágenes de tamaño 150x150 (PIXELES)*** y generará ***grupos de 32 imágenes (BATCH_SIZE)***, normalizadas y pudiendo realizar rotaciones (rotation_range) de 20 grados y zoom de hasta un 20% (zoom_range), pudiendo tambian modificar hasta un 20% de los píxeles de una foto (shear_range).
    
    + ***test_generator***: a partir de las carpeta de las imágenes de test, redimensionará las imágenes de tamaño 150x150 (PIXELES) y generará grupos de 32 imágenes (BATCH_SIZE), normalizadas. Dado que estas imágenes representan la "realidad", no realizaremos modificaciones de las mismas

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# CONSTANTES:
PIXELES = 150                   # Pixeles del alto y ancho de la imagen p.e-> (150,150)
BATCH_SIZE = 32                 # Número de imagenes por batch


# Definimos como modificar de manera aleatoria las imagenes (pixeles) de entrenamiento
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=20,
                                   horizontal_flip=True)

# Definimos como modificar las imagenes (pixeles) de test
#   rescale = normalizamos los pixeles
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Definimos como son nuestras imagenes de entrenamiento y test
train_generator = train_datagen.flow_from_directory(directory='./data/train',
                                                    target_size=(PIXELES, PIXELES),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(directory='./data/test',
                                                  target_size=(PIXELES, PIXELES),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical')

num_classes = train_generator.num_classes
print("Nº de Imagenes para entrenamiento: {}".format(train_generator.n))
print("Nº de Imagenes para test: {}".format(test_generator.n))
print("Nº de Clases a Clasificar: {} Clases".format(num_classes))

Found 18687 images belonging to 8 classes.
Found 4463 images belonging to 8 classes.
Nº de Imagenes para entrenamiento: 18687
Nº de Imagenes para test: 4463
Nº de Clases a Clasificar: 8 Clases


## 4.- Definimos el modelo de la red neuronal convolucional

* Definimos una red neuronal con la siguiente ***arquitectura***:

    1. Imagenes de Entrada 150 pixeles Ancho, 150 Pixeles de Alto, 3 Canales
    2. Capa Convolucional: 32 filtros, kernel (3x3), Función Activación RELU
    3. MaxPooling: Reducción de (2,2)
    4. Capa Convolucional: 64 filtros, kernel (3x3), Función Activación RELU
    5. MaxPooling: Reducción de (2,2)
    6. Capa Flatten: Capa de entrada del clasificador. Pasa cada Pixel a neurona
    7. Capa Oculta 1: 512 Neurona, Función Activación RELU
    8. Capa Oculta 2: 64 Neurona, Función Activación RELU
    9. Capa Salida: 6 Neurona (6 Clases), Función Activación SOFTMAX
    
    
* El modelo va a tener ***42 Millones de parámetros*** (exactamente 42.520.584 parámetros)


* Para la ***optimización de los parámetros*** de la red utilizaremos:

    + Función de perdida: ***categorical_crossentropy***
    + Optimizador: ***ADAM***
    + Métricas a monitorizar: Accuracy
    
    
#### Nota: La finalidad de este proyecto no es la de conseguir el mejor modelo posible de clasificación de los 8 tipos de imágenes del dataset, si no la de mostrar las diferencias de tiempo de entrenamiento que hay entre el uso de una CPU y GPU. Por ese motivo se ha definido una red neuronal con un número de parámetros (42M) lo suficientemente relevante como para ver las diferencias de tiempos de entrenamiento entre una CPU y GPU. Seguramente con una red neuronal menos compleja (con menos parámetros) se conseguiran mejores resultados de accuracy en la clasificación de estas imágenes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


# Definimos el modelo
model = Sequential()
model.add(Conv2D(filters=32,
                 kernel_size=(3, 3),
                 input_shape=(PIXELES, PIXELES, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64,
                 kernel_size=(3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# Imprimimos por pantalla la arquitectura de la red definida
print(model.summary())

# Compilamos el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 512)               4

## 5.- Entrenamos la red con la GPU


* Tanto para el entrenamiento del modelo con GPU y CPU usaremos:

    + epochs: ***5 epochs***
    + steps_per_epoch: Cada epoch tendrá ***1000 batches de 32 imágenes***
    + imágenes de entrenamiento: imagenes cargadas en la variable ***train_generator*** de la clase ImageDataGenerator
    + imágenes de test: tras cada epoch se validarán 320 imágenes (***10 batches*** *validation_steps* de ***32 imágenes***) cargadas en la variable *test_generator* de la clase ImageDataGenerator
    + workers: número de hilos (*12*) para el procesamiento en paralelo de la CPU. Para el caso del entrenamiento con GPU, será la CPU la encargada de pasar a la GPU los batches con las imágenes de entrenamiento.


In [5]:
# CONSTANTES:
NUM_EPOCHS = 5                  # Número de epochs
NUM_BATCHES_PER_EPOCH = 1000    # Número de Batches a realizar en cada EPOCH

# Ejecución con GPU
try:
    with tensorflow.device('/gpu:0'):
        print("### EJECUCIÓN CON GPU ###")
        model.fit(train_generator,
                  epochs=NUM_EPOCHS,
                  steps_per_epoch=NUM_BATCHES_PER_EPOCH,
                  validation_data=test_generator,
                  validation_steps=10,
                  workers=12,
                  verbose=1)
except Exception as e:
    print('WARNING: No es posible ejecutar con GPU: {}'.format(e))

### EJECUCIÓN CON GPU ###
Epoch 1/5
1000/1000 [==============================] - 40s 37ms/step - loss: 1.0500 - categorical_accuracy: 0.6237 - val_loss: 0.9117 - val_categorical_accuracy: 0.6656
Epoch 2/5
1000/1000 [==============================] - 42s 42ms/step - loss: 0.7225 - categorical_accuracy: 0.7492 - val_loss: 0.7006 - val_categorical_accuracy: 0.7500
Epoch 3/5
1000/1000 [==============================] - 44s 43ms/step - loss: 0.6291 - categorical_accuracy: 0.7859 - val_loss: 0.6797 - val_categorical_accuracy: 0.7625
Epoch 4/5
1000/1000 [==============================] - 44s 43ms/step - loss: 0.5530 - categorical_accuracy: 0.8082 - val_loss: 0.7137 - val_categorical_accuracy: 0.7625
Epoch 5/5
1000/1000 [==============================] - 44s 44ms/step - loss: 0.5133 - categorical_accuracy: 0.8259 - val_loss: 0.7788 - val_categorical_accuracy: 0.7125


## 6.- Entrenamos la red con la CPU

In [6]:

# Ejecución con CPU
try:
    with tensorflow.device('/cpu:0'):
        print("### EJECUCIÓN CON CPU ###")
        model.fit(train_generator,
                  epochs=NUM_EPOCHS,
                  steps_per_epoch=NUM_BATCHES_PER_EPOCH,
                  validation_data=test_generator,
                  validation_steps=10,
                  workers=12,
                  verbose=1)
except Exception as e:
    print('WARNING: No es posible ejecutar con CPU: {}'.format(e))

### EJECUCIÓN CON CPU ###
Epoch 1/5
1000/1000 [==============================] - 408s 407ms/step - loss: 0.4876 - categorical_accuracy: 0.8331 - val_loss: 0.8306 - val_categorical_accuracy: 0.7437
Epoch 2/5
1000/1000 [==============================] - 418s 417ms/step - loss: 0.4548 - categorical_accuracy: 0.8434 - val_loss: 0.6814 - val_categorical_accuracy: 0.7688
Epoch 3/5
1000/1000 [==============================] - 413s 413ms/step - loss: 0.4243 - categorical_accuracy: 0.8528 - val_loss: 1.0471 - val_categorical_accuracy: 0.7375
Epoch 4/5
1000/1000 [==============================] - 416s 416ms/step - loss: 0.4104 - categorical_accuracy: 0.8592 - val_loss: 0.6739 - val_categorical_accuracy: 0.8062
Epoch 5/5
1000/1000 [==============================] - 419s 418ms/step - loss: 0.4008 - categorical_accuracy: 0.8615 - val_loss: 0.6210 - val_categorical_accuracy: 0.7750
